In [2]:
import boto3
import json
import pandas as pd
s3 = boto3.resource("s3")
bucket = s3.Bucket('auto-shorts')

In [1]:
from dotenv import load_dotenv
load_dotenv('../../extractors/.env')

True

In [3]:
all_data = {}
for obj in bucket.objects.all():
    target = "tmp_data.json"
    if 'video_data.json' in obj.key:
        bucket.download_file(obj.key, target)
        with open('tmp_data.json', 'r') as f:
            vid_data = json.load(f)
            vid_path = '/'.join(obj.key.split('/')[:-1])
            vid_len = vid_data['most_watched_moments'][-1]['time_end_ms']/1000
            all_data[vid_path] = vid_len

In [4]:
df = pd.DataFrame(list(all_data.items()), columns=['s3_path', 'vid_len'])

In [5]:
df

,s3_path,vid_len
0,data/videos/17/UC1VU-FHvhbom5tPSCEB-e4g/-reCVT...,507.0
1,data/videos/17/UC1VU-FHvhbom5tPSCEB-e4g/2kMdsB...,522.0
2,data/videos/17/UC1VU-FHvhbom5tPSCEB-e4g/PBokg3...,571.0
3,data/videos/17/UC1VU-FHvhbom5tPSCEB-e4g/eIMuFo...,797.0
4,data/videos/17/UC1VU-FHvhbom5tPSCEB-e4g/sLJyFE...,440.0
...,...,...
181,data/videos/24/UCxOzbkk0bdVl6-tH1Fcajfg/nLeUvD...,1206.0
182,data/videos/24/UCxOzbkk0bdVl6-tH1Fcajfg/wdAva6...,1163.0
183,data/videos/24/UCxOzbkk0bdVl6-tH1Fcajfg/xYQ28e...,1559.0
184,data/videos/24/UCzskh68cVFMgzvqWrTGQ2uw/1UxPXd...,664.0


In [6]:
import sys
sys.path.append('../auto_shorts/upload/db/')
from utils import postgres_engine

In [7]:
from sqlalchemy import text


In [8]:
with postgres_engine.connect() as conn:
    df_vids = pd.read_sql_query(text('SELECT * FROM autoshorts.videos'), con=conn)[['s3_path', 'category_id']]

In [9]:
final_df = df.merge(df_vids, on='s3_path')

In [12]:
final_df.groupby('category_id')['vid_len'].agg(['min', 'max', 'std', 'mean', 'count', 'median']).T

category_id,17,20,22,23,24,26
min,371.000000,645.000000,282.000000,620.00000,219.000000,529.0
max,1173.000000,978.000000,1013.000000,1336.00000,2082.000000,529.0
std,162.301041,235.466558,271.106848,268.59818,418.564622,NaN
mean,597.821782,811.500000,672.384615,875.87500,1090.412698,529.0
count,101.000000,2.000000,13.000000,8.00000,63.000000,1.0
median,530.000000,811.500000,790.000000,822.50000,1047.000000,529.0
